In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
%matplotlib inline

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                       

# 1. Data acquisition and preparation

Get the data of the population of Paris

In [26]:
pop=pd.read_html('https://www.statista.com/statistics/1046193/population-by-district-arrondissements-paris-france/')[0]
pop.columns=['Neighborhood','Population in thousand']
pop

,Neighborhood,Population in thousand
0,1st arrondissement (Louvre),16338
1,2nd arrondissement (Bourse),20410
2,3rd arrondissement (Temple),35469
3,4th arrondissement (Hôtel-de-Ville),27795
4,5th arrondissement (Panthéon),59947
5,6th arrondissement (Luxembourg),41831
6,7th arrondissement (Palais-Bourbon),53415
7,8th arrondissement (Elysée),37053
8,9th arrondissement (Opéra),60235
9,10th arrondissement (Entrepôt),92660


Get the geographic data of Paris

In [27]:
paris=pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/0d3553c6-45c0-4b16-82be-5ef314437d3e',sep=";")
paris

,N_SQ_AR,C_AR,C_ARINSEE,L_AR,L_AROFF,N_SQ_CO,SURFACE,PERIMETRE,Geometry X Y,Geometry,OBJECTID,LONGUEUR
0,750000001,1,75101,1er Ardt,Louvre,750001537,1.824613e+06,6054.936862,"48.8625627018,2.33644336205","{""type"": ""Polygon"", ""coordinates"": [[[2.328007...",1,6054.680862
1,750000002,2,75102,2ème Ardt,Bourse,750001537,9.911537e+05,4554.104360,"48.8682792225,2.34280254689","{""type"": ""Polygon"", ""coordinates"": [[[2.351518...",2,4553.938764
2,750000017,17,75117,17ème Ardt,Batignolles-Monceau,750001537,5.668835e+06,10775.579516,"48.887326522,2.30677699057","{""type"": ""Polygon"", ""coordinates"": [[[2.295166...",17,10775.348915
3,750000014,14,75114,14ème Ardt,Observatoire,750001537,5.614877e+06,10317.483310,"48.8292445005,2.3265420442","{""type"": ""Polygon"", ""coordinates"": [[[2.333806...",14,10316.720258
4,750000020,20,75120,20ème Ardt,Ménilmontant,750001537,5.983446e+06,10704.940486,"48.8634605789,2.40118812928","{""type"": ""Polygon"", ""coordinates"": [[[2.412765...",20,10704.483774
5,750000007,7,75107,7ème Ardt,Palais-Bourbon,750001537,4.090057e+06,8099.424883,"48.8561744288,2.31218769148","{""type"": ""Polygon"", ""coordinates"": [[[2.320902...",7,8099.033151
6,750000011,11,75111,11ème Ardt,Popincourt,750001537,3.665442e+06,8282.011886,"48.8590592213,2.3800583082","{""type"": ""Polygon"", ""coordinates"": [[[2.396236...",11,8281.635288
7,750000013,13,75113,13ème Ardt,Gobelins,750001537,7.149311e+06,11546.546526,"48.8283880317,2.36227244042","{""type"": ""Polygon"", ""coordinates"": [[[2.374913...",13,11545.686986
8,750000004,4,75104,4ème Ardt,Hôtel-de-Ville,750001537,1.600586e+06,5420.908434,"48.8543414263,2.35762962032","{""type"": ""Polygon"", ""coordinates"": [[[2.368512...",4,5420.636779
9,750000008,8,75108,8ème Ardt,Élysée,750001537,3.880036e+06,7880.533268,"48.8727208374,2.3125540224","{""type"": ""Polygon"", ""coordinates"": [[[2.325836...",8,7880.260661


Rename some columns

In [28]:
paris.rename(columns={'L_AR':'Neighborhood Number','L_AROFF':'Name'},inplace=True)
paris

,N_SQ_AR,C_AR,C_ARINSEE,Neighborhood Number,Name,N_SQ_CO,SURFACE,PERIMETRE,Geometry X Y,Geometry,OBJECTID,LONGUEUR
0,750000001,1,75101,1er Ardt,Louvre,750001537,1.824613e+06,6054.936862,"48.8625627018,2.33644336205","{""type"": ""Polygon"", ""coordinates"": [[[2.328007...",1,6054.680862
1,750000002,2,75102,2ème Ardt,Bourse,750001537,9.911537e+05,4554.104360,"48.8682792225,2.34280254689","{""type"": ""Polygon"", ""coordinates"": [[[2.351518...",2,4553.938764
2,750000017,17,75117,17ème Ardt,Batignolles-Monceau,750001537,5.668835e+06,10775.579516,"48.887326522,2.30677699057","{""type"": ""Polygon"", ""coordinates"": [[[2.295166...",17,10775.348915
3,750000014,14,75114,14ème Ardt,Observatoire,750001537,5.614877e+06,10317.483310,"48.8292445005,2.3265420442","{""type"": ""Polygon"", ""coordinates"": [[[2.333806...",14,10316.720258
4,750000020,20,75120,20ème Ardt,Ménilmontant,750001537,5.983446e+06,10704.940486,"48.8634605789,2.40118812928","{""type"": ""Polygon"", ""coordinates"": [[[2.412765...",20,10704.483774
5,750000007,7,75107,7ème Ardt,Palais-Bourbon,750001537,4.090057e+06,8099.424883,"48.8561744288,2.31218769148","{""type"": ""Polygon"", ""coordinates"": [[[2.320902...",7,8099.033151
6,750000011,11,75111,11ème Ardt,Popincourt,750001537,3.665442e+06,8282.011886,"48.8590592213,2.3800583082","{""type"": ""Polygon"", ""coordinates"": [[[2.396236...",11,8281.635288
7,750000013,13,75113,13ème Ardt,Gobelins,750001537,7.149311e+06,11546.546526,"48.8283880317,2.36227244042","{""type"": ""Polygon"", ""coordinates"": [[[2.374913...",13,11545.686986
8,750000004,4,75104,4ème Ardt,Hôtel-de-Ville,750001537,1.600586e+06,5420.908434,"48.8543414263,2.35762962032","{""type"": ""Polygon"", ""coordinates"": [[[2.368512...",4,5420.636779
9,750000008,8,75108,8ème Ardt,Élysée,750001537,3.880036e+06,7880.533268,"48.8727208374,2.3125540224","{""type"": ""Polygon"", ""coordinates"": [[[2.325836...",8,7880.260661


The neighborhoods of Paris are identified by a number (1st to 20th arrondissement) or a name. Because we want to take the coordinates of Paris and merge with the population after, these two data frames have to have a common column, here I choose the neighborhood's name for the key column because the preparation is more simple. 

We see that in the population data frame, the neighborhood's number and name are merged together in the **Neighborhood** column, so we will extract the neighborhood's name from this column.

In [29]:
df=pd.DataFrame(pop['Neighborhood'].str.split(' arrondissement ').str[1])
df['Neighborhood'] = df['Neighborhood'].str.replace('(', '')
df['Neighborhood'] = df['Neighborhood'].str.replace(')', '')
pop['Name']=df
pop['Neighborhood']=pd.DataFrame(pop['Neighborhood'].str.split('(',1).str[0])
column=['Neighborhood','Name','Population in thousand']
pop=pop[column]
pop.head()

,Neighborhood,Name,Population in thousand
0,1st arrondissement,Louvre,16338
1,2nd arrondissement,Bourse,20410
2,3rd arrondissement,Temple,35469
3,4th arrondissement,Hôtel-de-Ville,27795
4,5th arrondissement,Panthéon,59947


We will need to use the coordinates (latitude and longitude) of each neighborhood to explore the venues, but these two values are merged together in the **Geometry X Y** column, it is necessary to separate them into two different columns

In [30]:
paris['Latitude']=pd.DataFrame(paris['Geometry X Y'].str.split(',').str[0])
paris['Longitude']=pd.DataFrame(paris['Geometry X Y'].str.split(',').str[1])
paris

,N_SQ_AR,C_AR,C_ARINSEE,Neighborhood Number,Name,N_SQ_CO,SURFACE,PERIMETRE,Geometry X Y,Geometry,OBJECTID,LONGUEUR,Latitude,Longitude
0,750000001,1,75101,1er Ardt,Louvre,750001537,1.824613e+06,6054.936862,"48.8625627018,2.33644336205","{""type"": ""Polygon"", ""coordinates"": [[[2.328007...",1,6054.680862,48.8625627018,2.33644336205
1,750000002,2,75102,2ème Ardt,Bourse,750001537,9.911537e+05,4554.104360,"48.8682792225,2.34280254689","{""type"": ""Polygon"", ""coordinates"": [[[2.351518...",2,4553.938764,48.8682792225,2.34280254689
2,750000017,17,75117,17ème Ardt,Batignolles-Monceau,750001537,5.668835e+06,10775.579516,"48.887326522,2.30677699057","{""type"": ""Polygon"", ""coordinates"": [[[2.295166...",17,10775.348915,48.887326522,2.30677699057
3,750000014,14,75114,14ème Ardt,Observatoire,750001537,5.614877e+06,10317.483310,"48.8292445005,2.3265420442","{""type"": ""Polygon"", ""coordinates"": [[[2.333806...",14,10316.720258,48.8292445005,2.3265420442
4,750000020,20,75120,20ème Ardt,Ménilmontant,750001537,5.983446e+06,10704.940486,"48.8634605789,2.40118812928","{""type"": ""Polygon"", ""coordinates"": [[[2.412765...",20,10704.483774,48.8634605789,2.40118812928
5,750000007,7,75107,7ème Ardt,Palais-Bourbon,750001537,4.090057e+06,8099.424883,"48.8561744288,2.31218769148","{""type"": ""Polygon"", ""coordinates"": [[[2.320902...",7,8099.033151,48.8561744288,2.31218769148
6,750000011,11,75111,11ème Ardt,Popincourt,750001537,3.665442e+06,8282.011886,"48.8590592213,2.3800583082","{""type"": ""Polygon"", ""coordinates"": [[[2.396236...",11,8281.635288,48.8590592213,2.3800583082
7,750000013,13,75113,13ème Ardt,Gobelins,750001537,7.149311e+06,11546.546526,"48.8283880317,2.36227244042","{""type"": ""Polygon"", ""coordinates"": [[[2.374913...",13,11545.686986,48.8283880317,2.36227244042
8,750000004,4,75104,4ème Ardt,Hôtel-de-Ville,750001537,1.600586e+06,5420.908434,"48.8543414263,2.35762962032","{""type"": ""Polygon"", ""coordinates"": [[[2.368512...",4,5420.636779,48.8543414263,2.35762962032
9,750000008,8,75108,8ème Ardt,Élysée,750001537,3.880036e+06,7880.533268,"48.8727208374,2.3125540224","{""type"": ""Polygon"", ""coordinates"": [[[2.325836...",8,7880.260661,48.8727208374,2.3125540224


Some neighborhoods' name have to be corrected to match the population data frame. It is the case of the rows 5 and 6, where *Élysée* must be corrected to *Elysée* and *Buttes-Montmartre* to *Butte-Montmartre*

In [53]:
try:
    paris.iloc[paris.Name[paris.Name == 'Élysée'].index.tolist()[0],4]='Elysée'    
except:
    print('No value found or value already replaced !')
    
try:
    paris.iloc[paris.Name[paris.Name == 'Buttes-Montmartre'].index.tolist()[0],4]='Butte-Montmartre'
except:
    print('No value found or value already replaced !')

paris

No value found or value already replaced !
No value found or value already replaced !


,N_SQ_AR,C_AR,C_ARINSEE,Neighborhood Number,Name,N_SQ_CO,SURFACE,PERIMETRE,Geometry X Y,Geometry,OBJECTID,LONGUEUR,Latitude,Longitude
0,750000001,1,75101,1er Ardt,Louvre,750001537,1.824613e+06,6054.936862,"48.8625627018,2.33644336205","{""type"": ""Polygon"", ""coordinates"": [[[2.328007...",1,6054.680862,48.8625627018,2.33644336205
1,750000002,2,75102,2ème Ardt,Bourse,750001537,9.911537e+05,4554.104360,"48.8682792225,2.34280254689","{""type"": ""Polygon"", ""coordinates"": [[[2.351518...",2,4553.938764,48.8682792225,2.34280254689
2,750000017,17,75117,17ème Ardt,Batignolles-Monceau,750001537,5.668835e+06,10775.579516,"48.887326522,2.30677699057","{""type"": ""Polygon"", ""coordinates"": [[[2.295166...",17,10775.348915,48.887326522,2.30677699057
3,750000014,14,75114,14ème Ardt,Observatoire,750001537,5.614877e+06,10317.483310,"48.8292445005,2.3265420442","{""type"": ""Polygon"", ""coordinates"": [[[2.333806...",14,10316.720258,48.8292445005,2.3265420442
4,750000020,20,75120,20ème Ardt,Ménilmontant,750001537,5.983446e+06,10704.940486,"48.8634605789,2.40118812928","{""type"": ""Polygon"", ""coordinates"": [[[2.412765...",20,10704.483774,48.8634605789,2.40118812928
5,750000007,7,75107,7ème Ardt,Palais-Bourbon,750001537,4.090057e+06,8099.424883,"48.8561744288,2.31218769148","{""type"": ""Polygon"", ""coordinates"": [[[2.320902...",7,8099.033151,48.8561744288,2.31218769148
6,750000011,11,75111,11ème Ardt,Popincourt,750001537,3.665442e+06,8282.011886,"48.8590592213,2.3800583082","{""type"": ""Polygon"", ""coordinates"": [[[2.396236...",11,8281.635288,48.8590592213,2.3800583082
7,750000013,13,75113,13ème Ardt,Gobelins,750001537,7.149311e+06,11546.546526,"48.8283880317,2.36227244042","{""type"": ""Polygon"", ""coordinates"": [[[2.374913...",13,11545.686986,48.8283880317,2.36227244042
8,750000004,4,75104,4ème Ardt,Hôtel-de-Ville,750001537,1.600586e+06,5420.908434,"48.8543414263,2.35762962032","{""type"": ""Polygon"", ""coordinates"": [[[2.368512...",4,5420.636779,48.8543414263,2.35762962032
9,750000008,8,75108,8ème Ardt,Elysée,750001537,3.880036e+06,7880.533268,"48.8727208374,2.3125540224","{""type"": ""Polygon"", ""coordinates"": [[[2.325836...",8,7880.260661,48.8727208374,2.3125540224


Once all data are corrected prepared, now we will merge the population and coordinates together into a new data frame named **data**

In [54]:
data=pop.merge(paris[['Name','Latitude','Longitude']],on='Name')
#data.astype({'Latitude': 'float','Longitude': 'float'},inplace=True)
data['Latitude'] = pd.to_numeric(data['Latitude'])
data['Longitude'] = pd.to_numeric(data['Longitude'])
data

,Neighborhood,Name,Population in thousand,Latitude,Longitude
0,1st arrondissement,Louvre,16338,48.862563,2.336443
1,2nd arrondissement,Bourse,20410,48.868279,2.342803
2,3rd arrondissement,Temple,35469,48.862872,2.360001
3,4th arrondissement,Hôtel-de-Ville,27795,48.854341,2.357630
4,5th arrondissement,Panthéon,59947,48.844443,2.350715
5,6th arrondissement,Luxembourg,41831,48.849130,2.332898
6,7th arrondissement,Palais-Bourbon,53415,48.856174,2.312188
7,8th arrondissement,Elysée,37053,48.872721,2.312554
8,9th arrondissement,Opéra,60235,48.877164,2.337458
9,10th arrondissement,Entrepôt,92660,48.876130,2.360728


# 2. Exploration Data Analysis

First, we visualize the population and center of each neighborhood on a choropleth map

In [55]:
address = 'Paris'

geolocator = Nominatim(user_agent="bay")
location = geolocator.geocode(address)
paris_latitude = location.latitude
paris_longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,paris_latitude, paris_longitude))

The geograpical coordinate of Paris are 48.8566969, 2.3514616.


In [56]:
#link: https://www.data.gouv.fr/fr/datasets/arrondissements-1/#_
paris_geo = f'https://www.data.gouv.fr/fr/datasets/r/4765fe48-35fd-4536-b029-4727380ce23c'
paris_map = folium.Map(location=[paris_latitude,paris_longitude], zoom_start=12) #, tiles='Mapbox Bright')

paris_map.choropleth(
    geo_data=paris_geo,
    data=data,
    columns=['Name', 'Population in thousand'],
    key_on='feature.properties.l_aroff',
    #threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Populations of Paris'
)


for lat, lng, neigh in zip(data['Latitude'], data['Longitude'], data['Neighborhood']):
    label = neigh
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(paris_map)  
    
paris_map

Then, we will the Foursquare API to explore the venues in Paris

In [59]:
CLIENT_ID = '4MUHPKIL44EV114RAA4VEAQAOP3ZG0MPWSDSYAHU502RUH0A' # your Foursquare ID
CLIENT_SECRET = 'CYHV2HQXBAFGQV4MZGEYXMA2LQKK0DWKGE2BNI32WBZ2QFUJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4MUHPKIL44EV114RAA4VEAQAOP3ZG0MPWSDSYAHU502RUH0A
CLIENT_SECRET:CYHV2HQXBAFGQV4MZGEYXMA2LQKK0DWKGE2BNI32WBZ2QFUJ


In [60]:
def getNearbyVenues(names, populations,latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, pop,lat, lng in zip(names, populations, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            1000, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,            
            lat, 
            lng,
            pop,
            v['venue']['id'],
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude',
                  'Population in thousand',
                  'Venue ID',
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [61]:
venues = getNearbyVenues(names=data['Neighborhood'],
                               populations=data['Population in thousand'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                  )

1st arrondissement 
2nd arrondissement 
3rd arrondissement 
4th arrondissement 
5th arrondissement 
6th arrondissement 
7th arrondissement 
8th arrondissement 
9th arrondissement 
10th arrondissement 
11th arrondissement 
12th arrondissement 
13th arrondissement 
14th arrondissement 
15th arrondissement 
16th arrondissement 
17th arrondissement 
18th arrondissement 
19th arrondissement 
20th arrondissement 


In [62]:
print(venues.shape)
venues.head(10)

(1838, 9)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Population in thousand,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,1st arrondissement,48.862563,2.336443,16338,4adcda10f964a520af3521e3,Musée du Louvre,48.860847,2.336440,Art Museum
1,1st arrondissement,48.862563,2.336443,16338,4adcda09f964a520ed3321e3,Palais Royal,48.863236,2.337127,Historic Site
2,1st arrondissement,48.862563,2.336443,16338,4b48b4b0f964a5207c5326e3,Comédie-Française,48.863088,2.336612,Theater
3,1st arrondissement,48.862563,2.336443,16338,4f6dabf5003944083fe0002e,Vestige de la Forteresse du Louvre,48.861577,2.333508,Historic Site
4,1st arrondissement,48.862563,2.336443,16338,571fe799498ef36a9fd63f1a,La Clef Louvre Paris,48.863977,2.336140,Hotel
5,1st arrondissement,48.862563,2.336443,16338,5072efe7e4b0c34b5146e7fd,Cour Napoléon,48.861172,2.335088,Plaza
6,1st arrondissement,48.862563,2.336443,16338,4ba8b650f964a520f5e839e3,Jardin du Palais Royal,48.864941,2.337728,Garden
7,1st arrondissement,48.862563,2.336443,16338,4b071505f964a520dcf622e3,Place du Palais Royal,48.862523,2.336688,Plaza
8,1st arrondissement,48.862563,2.336443,16338,4c079d740ed3c928b6be797d,Cour Carrée du Louvre,48.860360,2.338543,Pedestrian Plaza
9,1st arrondissement,48.862563,2.336443,16338,57421b43498eb3b82c128e67,LouLou,48.862804,2.333500,Italian Restaurant


In [63]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Population in thousand,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,,
10th arrondissement,100,100,100,100,100,100,100,100
11th arrondissement,100,100,100,100,100,100,100,100
12th arrondissement,19,19,19,19,19,19,19,19
13th arrondissement,100,100,100,100,100,100,100,100
14th arrondissement,100,100,100,100,100,100,100,100
15th arrondissement,100,100,100,100,100,100,100,100
16th arrondissement,62,62,62,62,62,62,62,62
17th arrondissement,100,100,100,100,100,100,100,100
18th arrondissement,100,100,100,100,100,100,100,100


In [64]:
print('There are {} uniques categories.'.format(len(venues['Venue Category'].unique())))

There are 222 uniques categories.


In [65]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhood'] = venues['Neighborhood']


# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1]) #Create list of columnseorganize the columns so that the Neighborhood column is the first one
onehot = onehot[fixed_columns]
#onehot[['Venue Latitude','Venue Longitude']] = venues[['Venue Latitude','Venue Longitude']]

print(onehot.shape)
onehot.head()

(1838, 223)


,Neighborhood,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,1st arrondissement,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1st arrondissement,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1st arrondissement,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1st arrondissement,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1st arrondissement,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
onehot.shape

(1838, 223)

In [67]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped.head()

,Neighborhood,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Vineyard,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,10th arrondissement,0.00,0.02,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,...,0.02,0.00,0.00,0.0,0.01,0.03,0.0,0.0,0.000000,0.000000
1,11th arrondissement,0.01,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.01,...,0.00,0.01,0.01,0.0,0.04,0.00,0.0,0.0,0.000000,0.000000
2,12th arrondissement,0.00,0.00,0.0,0.0,0.0,0.052632,0.0,0.0,0.00,...,0.00,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.052632,0.052632
3,13th arrondissement,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,...,0.00,0.00,0.16,0.0,0.00,0.00,0.0,0.0,0.000000,0.000000
4,14th arrondissement,0.00,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,...,0.02,0.00,0.04,0.0,0.01,0.01,0.0,0.0,0.000000,0.000000


In [68]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [69]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,10th arrondissement,French Restaurant,Coffee Shop,Italian Restaurant,Bakery,Pizza Place,Japanese Restaurant,Breakfast Spot,Asian Restaurant,Cocktail Bar,Wine Shop
1,11th arrondissement,French Restaurant,Bar,Restaurant,Cocktail Bar,Bistro,Wine Bar,Italian Restaurant,Café,Moroccan Restaurant,Coffee Shop
2,12th arrondissement,Lake,French Restaurant,Zoo Exhibit,Track,History Museum,Zoo,Italian Restaurant,Japanese Restaurant,Monument / Landmark,Pool
3,13th arrondissement,Vietnamese Restaurant,Thai Restaurant,Asian Restaurant,French Restaurant,Chinese Restaurant,Hotel,Bakery,Japanese Restaurant,Bistro,Gym / Fitness Center
4,14th arrondissement,French Restaurant,Hotel,Italian Restaurant,Vietnamese Restaurant,Bar,Japanese Restaurant,Bakery,Pizza Place,Sushi Restaurant,Bistro


In [70]:
# set number of clusters
kclusters = 5

grouped_clustering = grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 3, 4, 1, 2, 2, 2, 2, 3, 3, 0, 3, 0, 0, 2, 2, 0, 2, 0, 2],
      dtype=int32)

In [71]:
kmeans.labels_.shape

(20,)

In [72]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

merged = data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

merged.head(10) # check the last columns!

,Neighborhood,Name,Population in thousand,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1st arrondissement,Louvre,16338,48.862563,2.336443,0,French Restaurant,Hotel,Japanese Restaurant,Plaza,Italian Restaurant,Art Museum,Historic Site,Wine Bar,Coffee Shop,Pastry Shop
1,2nd arrondissement,Bourse,20410,48.868279,2.342803,0,French Restaurant,Hotel,Wine Bar,Cocktail Bar,Cheese Shop,Italian Restaurant,Japanese Restaurant,Bookstore,Pedestrian Plaza,Boutique
2,3rd arrondissement,Temple,35469,48.862872,2.360001,0,Art Gallery,French Restaurant,Wine Bar,Hotel,Cocktail Bar,Italian Restaurant,Clothing Store,Bookstore,Bakery,Sandwich Place
3,4th arrondissement,Hôtel-de-Ville,27795,48.854341,2.357630,2,French Restaurant,Plaza,Park,Gourmet Shop,Wine Bar,Japanese Restaurant,Garden,Cocktail Bar,Ice Cream Shop,Bakery
4,5th arrondissement,Panthéon,59947,48.844443,2.350715,2,French Restaurant,Bakery,Plaza,Japanese Restaurant,Italian Restaurant,Indie Movie Theater,Museum,Hotel,Café,Coffee Shop
5,6th arrondissement,Luxembourg,41831,48.849130,2.332898,0,French Restaurant,Hotel,Plaza,Italian Restaurant,Seafood Restaurant,Garden,Wine Bar,Bistro,Indie Movie Theater,Cupcake Shop
6,7th arrondissement,Palais-Bourbon,53415,48.856174,2.312188,2,French Restaurant,Hotel,Plaza,Historic Site,Coffee Shop,Garden,History Museum,Cocktail Bar,Italian Restaurant,Café
7,8th arrondissement,Elysée,37053,48.872721,2.312554,0,Hotel,French Restaurant,Boutique,Garden,Clothing Store,Women's Store,Art Gallery,Theater,Pastry Shop,Park
8,9th arrondissement,Opéra,60235,48.877164,2.337458,2,French Restaurant,Hotel,Wine Bar,Plaza,Italian Restaurant,Cocktail Bar,Concert Hall,Bistro,Chocolate Shop,Cheese Shop
9,10th arrondissement,Entrepôt,92660,48.876130,2.360728,2,French Restaurant,Coffee Shop,Italian Restaurant,Bakery,Pizza Place,Japanese Restaurant,Breakfast Spot,Asian Restaurant,Cocktail Bar,Wine Shop


In [73]:
# create map
map_clusters = folium.Map(location=[paris_latitude,paris_longitude], zoom_start=12) #, tiles='Mapbox Bright')

map_clusters.choropleth(
    geo_data=paris_geo,
    data=data,
    columns=['Name', 'Population in thousand'],
    key_on='feature.properties.l_aroff',
    #threshold_scale=threshold_scale,
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Populations of Paris'
)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [74]:
merged.loc[merged['Cluster Labels'] == 0, merged.columns[[0] + list(range(5, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1st arrondissement,0,French Restaurant,Hotel,Japanese Restaurant,Plaza,Italian Restaurant,Art Museum,Historic Site,Wine Bar,Coffee Shop,Pastry Shop
1,2nd arrondissement,0,French Restaurant,Hotel,Wine Bar,Cocktail Bar,Cheese Shop,Italian Restaurant,Japanese Restaurant,Bookstore,Pedestrian Plaza,Boutique
2,3rd arrondissement,0,Art Gallery,French Restaurant,Wine Bar,Hotel,Cocktail Bar,Italian Restaurant,Clothing Store,Bookstore,Bakery,Sandwich Place
5,6th arrondissement,0,French Restaurant,Hotel,Plaza,Italian Restaurant,Seafood Restaurant,Garden,Wine Bar,Bistro,Indie Movie Theater,Cupcake Shop
7,8th arrondissement,0,Hotel,French Restaurant,Boutique,Garden,Clothing Store,Women's Store,Art Gallery,Theater,Pastry Shop,Park


In [75]:
merged.loc[merged['Cluster Labels'] == 1, merged.columns[[0] + list(range(5, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,13th arrondissement,1,Vietnamese Restaurant,Thai Restaurant,Asian Restaurant,French Restaurant,Chinese Restaurant,Hotel,Bakery,Japanese Restaurant,Bistro,Gym / Fitness Center


In [76]:
merged.loc[merged['Cluster Labels'] == 2, merged.columns[[0] + list(range(5, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,4th arrondissement,2,French Restaurant,Plaza,Park,Gourmet Shop,Wine Bar,Japanese Restaurant,Garden,Cocktail Bar,Ice Cream Shop,Bakery
4,5th arrondissement,2,French Restaurant,Bakery,Plaza,Japanese Restaurant,Italian Restaurant,Indie Movie Theater,Museum,Hotel,Café,Coffee Shop
6,7th arrondissement,2,French Restaurant,Hotel,Plaza,Historic Site,Coffee Shop,Garden,History Museum,Cocktail Bar,Italian Restaurant,Café
8,9th arrondissement,2,French Restaurant,Hotel,Wine Bar,Plaza,Italian Restaurant,Cocktail Bar,Concert Hall,Bistro,Chocolate Shop,Cheese Shop
9,10th arrondissement,2,French Restaurant,Coffee Shop,Italian Restaurant,Bakery,Pizza Place,Japanese Restaurant,Breakfast Spot,Asian Restaurant,Cocktail Bar,Wine Shop
13,14th arrondissement,2,French Restaurant,Hotel,Italian Restaurant,Vietnamese Restaurant,Bar,Japanese Restaurant,Bakery,Pizza Place,Sushi Restaurant,Bistro
14,15th arrondissement,2,French Restaurant,Italian Restaurant,Hotel,Korean Restaurant,Bakery,Japanese Restaurant,Persian Restaurant,Coffee Shop,Thai Restaurant,Lebanese Restaurant
15,16th arrondissement,2,French Restaurant,Bakery,Park,Italian Restaurant,Plaza,Bike Rental / Bike Share,Lake,Pool,Chinese Restaurant,Garden
16,17th arrondissement,2,French Restaurant,Italian Restaurant,Hotel,Bakery,Restaurant,Pastry Shop,Park,Turkish Restaurant,Bagel Shop,Mediterranean Restaurant


In [77]:
merged.loc[merged['Cluster Labels'] == 3, merged.columns[[0] + list(range(5, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,11th arrondissement,3,French Restaurant,Bar,Restaurant,Cocktail Bar,Bistro,Wine Bar,Italian Restaurant,Café,Moroccan Restaurant,Coffee Shop
17,18th arrondissement,3,French Restaurant,Bar,Bistro,Pizza Place,Café,Restaurant,Plaza,Middle Eastern Restaurant,Italian Restaurant,Art Gallery
18,19th arrondissement,3,French Restaurant,Bar,Café,Bistro,Concert Hall,Italian Restaurant,Pool,Scenic Lookout,Restaurant,Beer Bar
19,20th arrondissement,3,Bar,French Restaurant,Bakery,Café,Theater,Bistro,Bagel Shop,Plaza,Bookstore,Music Venue


In [78]:
merged.loc[merged['Cluster Labels'] == 4, merged.columns[[0] + list(range(5, merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,12th arrondissement,4,Lake,French Restaurant,Zoo Exhibit,Track,History Museum,Zoo,Italian Restaurant,Japanese Restaurant,Monument / Landmark,Pool


In [79]:
# one hot encoding
name_venues = pd.get_dummies(venues[['Venue']], prefix="", prefix_sep="")
name_venues.shape

(1838, 1675)

Now, I want to retrieve only the Vietnamese, Chinese and Thai restaurants in order to visualize them

In [102]:
AsianRestaurant=venues[venues['Venue Category'].isin(['Vietnamese Restaurant','Chinese Restaurant','Thai Restaurant'])]
AsianRestaurant.reset_index(inplace = True)
print(AsianRestaurant.shape)
AsianRestaurant.head()

(61, 10)


,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Population in thousand,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,136,2nd arrondissement,48.868279,2.342803,20410,54479500498ec5f5c5a9e8d0,Kapunka,48.866226,2.347763,Thai Restaurant
1,220,3rd arrondissement,48.862872,2.360001,35469,4b4b9831f964a5205fa126e3,Taing Song-Heng,48.864701,2.356888,Vietnamese Restaurant
2,245,3rd arrondissement,48.862872,2.360001,35469,587c9bbf286804584a1f28c6,Banoï,48.864156,2.366739,Vietnamese Restaurant
3,333,4th arrondissement,48.854341,2.357630,27795,4c6eb67006ed6dcb107fa722,Au Petit Thaï,48.855803,2.360159,Thai Restaurant
4,336,4th arrondissement,48.854341,2.357630,27795,4b6839fbf964a520e46c2be3,Thai Spices,48.852733,2.360359,Thai Restaurant


It will be interesting to see the rating of each restaurant, which is a good indicator of the custumor appreciation. This is very important for the understanding of the concurrence in an area. I use the Foursquare API to get this information. However, there is an obstacle here: the number of requests is limited to 50 per day, while there are 60 restaurants to be explored. To overcome this limitation, I take rating of the first 30 restaurants for the first day and the rest the following day. 

In [81]:
rating_list_0_to_29=[8.7,
8.7,
9,
8,
8,
8.3,
8.1,
8.7,
8.7,
8.9,
8.2,
8,
8.2,
8.2,
8.3,
7.8,
7.9,
7.8,
7.7,
7.5,
7.9,
8.5,
8.4,
8.7,
8,
8.2,
7.5,
7.4,
7.5,
8.3,
]

In [82]:
rating_list=[]
for venue_id in AsianRestaurant['Venue ID'][30:60]:
    url2='https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(venue_id,CLIENT_ID,CLIENT_SECRET,VERSION)
    rating = requests.get(url2).json()["response"]['venue']['rating'] 
    rating_list.append(rating)

KeyError: 'venue'

In [83]:
rating_list_30_to_59 = [7.5,
 7.8,
 8.0,
 7.9,
 8.4,
 7.3,
 7.3,
 7.7,
 7.3,
 7.5,
 8.2,
 7.2,
 7.3,
 7.3,
 7.9,
 7.2,
 7.4,
 8.4,
 8.1,
 7.4,
 7.4,
 6.9,
 7.8,
 7.8,
 8.1,
 6.7,
 8.3,
 8.1,
 7.8,
 7.6]

Merge the two list together and transform into data frame:

In [84]:
for element in rating_list_30_to_59:
    rating_list_0_to_29.append(element)
rating_list_full=rating_list_0_to_29

In [87]:
rating_df=pd.DataFrame(rating_list_full)
rating_df.columns=['Rating']  
rating_df.head()

,Rating
0,8.7
1,8.7
2,9.0
3,8.0
4,8.0


Classify the rating into 3 categories:
1. Good
2. Normal
3. Not good

In [105]:
ranking=[]
for rat in rating_df.Rating:
    if rat>=8.5: 
        ranking.append(1)
    elif rat>=7.5:
        ranking.append(2)
    else:
        ranking.append(3)
        
ranking=pd.DataFrame(ranking)
ranking.head()

,0
0,1
1,1
2,1
3,2
4,2


In [113]:
AsianRestaurant['Rating']=rating_df
AsianRestaurant['Ranking']=ranking
AsianRestaurant.dropna(inplace=True)
AsianRestaurant['Ranking'].astype(int)
AsianRestaurant

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org

,index,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Population in thousand,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category,Rating,Ranking
0,136,2nd arrondissement,48.868279,2.342803,20410,54479500498ec5f5c5a9e8d0,Kapunka,48.866226,2.347763,Thai Restaurant,8.7,1
1,220,3rd arrondissement,48.862872,2.360001,35469,4b4b9831f964a5205fa126e3,Taing Song-Heng,48.864701,2.356888,Vietnamese Restaurant,8.7,1
2,245,3rd arrondissement,48.862872,2.360001,35469,587c9bbf286804584a1f28c6,Banoï,48.864156,2.366739,Vietnamese Restaurant,9.0,1
3,333,4th arrondissement,48.854341,2.357630,27795,4c6eb67006ed6dcb107fa722,Au Petit Thaï,48.855803,2.360159,Thai Restaurant,8.0,2
4,336,4th arrondissement,48.854341,2.357630,27795,4b6839fbf964a520e46c2be3,Thai Spices,48.852733,2.360359,Thai Restaurant,8.0,2
5,422,5th arrondissement,48.844443,2.350715,59947,4b6b21c2f964a520e7f42be3,Foyer Vietnam,48.842187,2.351987,Vietnamese Restaurant,8.3,2
6,493,5th arrondissement,48.844443,2.350715,59947,4cd1aed12fd9b60cd25988eb,Au Vietnam,48.838562,2.351086,Vietnamese Restaurant,8.1,2
7,495,5th arrondissement,48.844443,2.350715,59947,5522d0a3498eaa1b74f88c64,Chez Neung,48.837563,2.354139,Thai Restaurant,8.7,1
8,725,8th arrondissement,48.872721,2.312554,37053,5237540711d21573dcd264fd,Tuk Tuk Thaï,48.875462,2.317854,Thai Restaurant,8.7,1
9,905,10th arrondissement,48.876130,2.360728,92660,55f162d1498edec075e8c3d2,Street Bangkok Local Food,48.877770,2.365204,Thai Restaurant,8.9,1


In [114]:
color_code=['green','blue','red']
color_code_fill=['#c5eba0','#3186cc','#f0a7a3']

In [115]:
paris_geo = f'https://www.data.gouv.fr/fr/datasets/r/4765fe48-35fd-4536-b029-4727380ce23c'
paris_map = folium.Map(location=[paris_latitude,paris_longitude], zoom_start=12)

paris_map.choropleth(
    geo_data=paris_geo,
    data=data,
    columns=['Name', 'Population in thousand'],
    key_on='feature.properties.l_aroff',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Populations of Paris'
)

for lat, lng, name, cate, rating, rank in zip(AsianRestaurant['Venue Latitude'], AsianRestaurant['Venue Longitude'], AsianRestaurant['Venue'],AsianRestaurant['Venue Category'],AsianRestaurant['Rating'],AsianRestaurant['Ranking']):
    label = name + ", " + str(rating)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color=color_code[rank-1],
        fill=True,
        fill_color=color_code_fill[rank-1],
        fill_opacity=0.7,
        parse_html=False).add_to(paris_map)  

paris_map